In [2]:
import base64
import boto3
import io
import json
from PIL import Image
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel


from config import IMAGE_TEST

In [11]:
role = get_execution_role()

pytorch_model = PyTorchModel(model_data='s3://weights-captcha-model/model.tar.gz', role=role,
                             entry_point='inference.py', framework_version="1.7.1",
                             py_version="py3",
                             )

In [12]:
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)
endpoint = predictor.endpoint
endpoint

--------!

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'pytorch-inference-2021-12-22-21-46-46-651'

In [13]:
image = Image.open(IMAGE_TEST).convert("RGB")
#bytesarray = bytes(image.tobytes())
#image_data = Image.open(io.BytesIO(image.tobytes()))
#image.show()

buffered = io.BytesIO()
image.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue())

payload = {"image": img_str.decode()}

In [14]:
payload

{'image': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAAyAMgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD0EU4U0U8UAeVeK/8ARPjf4buBwJYo0z7lpF/qK9ZFeTfFDNl418J6mwIijnXe+OBtkVsfkTXrIoAcKeK848SfFnT9Mu2sNFtm1W8BwfLzsU+mQMt+FZug/GVpdVWy8Q6cLEOwUSIG+QnpuB5x70AetinCszSNb

In [15]:
payload = json.dumps(payload)

runtime = boto3.client('sagemaker-runtime')

response = runtime.invoke_endpoint(
    EndpointName=endpoint,
    ContentType='application/json',
    Body=payload)

result = json.loads(response['Body'].read().decode()) 

In [16]:
print(result)

[[[12.823210716247559, -3.3536620140075684, -3.4441747665405273, -2.4756901264190674, -3.407151222229004, -3.9168782234191895, -2.047616958618164, -3.2130579948425293, -2.473849058151245, -1.6855981349945068, -2.3256077766418457, -3.5470213890075684, -1.7720228433609009, -2.0232253074645996, -1.396254539489746, -1.7755459547042847, -3.3379015922546387, -1.4763269424438477, -3.255138397216797, -2.4083404541015625]], [[13.559450149536133, -3.4233756065368652, -3.7279975414276123, -2.7007648944854736, -3.695909023284912, -4.2554144859313965, -2.2338621616363525, -3.3523306846618652, -2.677553653717041, -1.6076420545578003, -2.3442869186401367, -3.4424057006835938, -1.989740014076233, -2.0882837772369385, -1.7355692386627197, -2.0524632930755615, -3.341923236846924, -1.5665390491485596, -3.313802719116211, -2.568237543106079]], [[13.56861686706543, -3.406442403793335, -3.720794439315796, -2.725484848022461, -3.703204393386841, -4.29093074798584, -2.2275869846343994, -3.344917058944702, -2.

### Eliminar Endpoint

In [10]:
predictor.delete_endpoint()